### **Quickstart: Compare runs, choose a model, deploy it to REST API**

- run a hyperparameter sweep on a training script
- compare the results of the runs in the mlflow UI
- choose the best run and register it as a model
- deploy the model to a REST API
- build a container image suitable for deployment to a cloud platform

In [1]:
import keras
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

2026-01-12 22:22:10.085638: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-12 22:22:10.086903: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-12 22:22:10.236644: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-12 22:22:12.990386: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off,

In [5]:
# load the dataset
data = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow-example/refs/heads/master/wine-quality.csv")
data.head(3)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6


In [9]:
# split the data into train and test sets
train, test = train_test_split(data, test_size=0.2, random_state=42)

train_x = train.drop(['quality'], axis=1).values # features for train set
train_y = train['quality'].values.ravel() # target for train set

# test data
test_x = test.drop(['quality'], axis=1).values # features for test set
test_y = test['quality'].values.ravel() # target for test set

# train and validation data
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y,
                                                      test_size=0.2, 
                                                      random_state=42)

# infer signature
signature = infer_signature(train_x, train_y)

In [10]:
# ANN MODEL
def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
    # define model architecture
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)

    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64, activation='relu'),
            keras.layers.Dense(1)
        ])
    
    # compile the model
    model.compile(optimizer=keras.optimizers.SGD(
                  learning_rate=params['lr'],
                  momentum=params['momentum']),
                  loss="mean_squared_error",
                  metrics=[keras.metrics.RootMeanSquaredError()])
    
    # train the model with lr, momentum params with MLFlow tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x, train_y, validation_data=(valid_x, valid_y),
                  epochs=epochs,
                  batch_size=64)
        
        # evaluate the model
        eval_result = model.evaluate(valid_x, valid_y, batch_size=64)

        eval_rmse = eval_result[1]

        # log the parameters and metrics
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        # log the model
        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"loss": eval_rmse,
                "status": STATUS_OK,
                "model": model}

In [11]:
def objective(params):
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y
    )

    return result

In [15]:
space = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 0.1)
}

In [16]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    # conduct hyperparam search using hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    # fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x['loss'])[0]

    # log the best parameters, loss and model
    mlflow.log_params(best)
    mlflow.log_metric("eval rmse", best_run["loss"])

  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]

Epoch 1/3                                            

 1/49 ━━━━━━━━━━━━━━━━━━━━ 41s 867ms/step - loss: 37.0933 - root_mean_squared_error: 6.0904
 9/49 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 38.1396 - root_mean_squared_error: 6.1755   
20/49 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 37.7465 - root_mean_squared_error: 6.1436
30/49 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 37.3468 - root_mean_squared_error: 6.1109
39/49 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 37.0334 - root_mean_squared_error: 6.0851
48/49 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 36.7146 - root_mean_squared_error: 6.0587
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 34.9476 - root_mean_squared_error: 5.9117 - val_loss: 31.8088 - val_root_mean_squared_error: 5.6399

Epoch 2/3                                            

 1/49 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 31.3757 - root_mean_squared_error: 5.6014
17/49 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 30.6662 - root_mean_squared_error: 5.5377 
26/49 ━━━━━━━━━━━━━━━

2026/01/12 22:56:40 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/3                                                                     

 1/49 ━━━━━━━━━━━━━━━━━━━━ 21s 438ms/step - loss: 35.9197 - root_mean_squared_error: 5.9933
 9/49 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 33.7718 - root_mean_squared_error: 5.8103   
16/49 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 32.3103 - root_mean_squared_error: 5.6816
24/49 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 30.8250 - root_mean_squared_error: 5.5468
34/49 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 29.0946 - root_mean_squared_error: 5.3840
45/49 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 27.3726 - root_mean_squared_error: 5.2156
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 20.0088 - root_mean_squared_error: 4.4731 - val_loss: 9.7931 - val_root_mean_squared_error: 3.1294

Epoch 2/3                                                                     

 1/49 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - loss: 10.7452 - root_mean_squared_error: 3.2780
 9/49 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 9.4616 - root_m

2026/01/12 22:56:57 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/3                                                                      

 1/49 ━━━━━━━━━━━━━━━━━━━━ 45s 938ms/step - loss: 33.9539 - root_mean_squared_error: 5.8270
20/49 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9.6302 - root_mean_squared_error: 2.9447    
38/49 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.6255 - root_mean_squared_error: 2.4071
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 2.3093 - root_mean_squared_error: 1.5196 - val_loss: 0.8410 - val_root_mean_squared_error: 0.9170

Epoch 2/3                                                                      

 1/49 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.8058 - root_mean_squared_error: 0.8976
20/49 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7765 - root_mean_squared_error: 0.8809 
41/49 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7570 - root_mean_squared_error: 0.8698
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6892 - root_mean_squared_error: 0.8302 - val_loss: 0.5530 - val_root_mean_squared_error: 0.7436

Epoch 3/

2026/01/12 22:57:09 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/3                                                                      

 1/49 ━━━━━━━━━━━━━━━━━━━━ 30s 625ms/step - loss: 33.9525 - root_mean_squared_error: 5.8269
23/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 33.3324 - root_mean_squared_error: 5.7733   
41/49 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 33.1348 - root_mean_squared_error: 5.7562
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 32.1437 - root_mean_squared_error: 5.6695 - val_loss: 29.9361 - val_root_mean_squared_error: 5.4714

Epoch 2/3                                                                     

 1/49 ━━━━━━━━━━━━━━━━━━━━ 3s 74ms/step - loss: 29.0785 - root_mean_squared_error: 5.3924
 9/49 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 28.9029 - root_mean_squared_error: 5.3761 
17/49 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 28.8935 - root_mean_squared_error: 5.3752
25/49 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 28.8347 - root_mean_squared_error: 5.3698
35/49 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 28.7105 - ro

2026/01/12 22:57:22 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



100%|██████████| 4/4 [00:56<00:00, 14.12s/trial, best loss: 0.741887092590332]
